**TEXT** **CLASSIFICATION**

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
pip install transformers

     |████████████████████████████████| 675kB 3.5MB/s 
     |████████████████████████████████| 890kB 15.1MB/s 
     |████████████████████████████████| 3.8MB 19.8MB/s 
     |████████████████████████████████| 1.1MB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4d97058a92fbc2c595a5dd582dcabb1ec191974fa7a1c373d4f87b4ec1f9442e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers  as ppb 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#We can use BERT but here I am using DistillBERT because BERT requires more RAM then available in the colab,but to use BERT just uncomment the next line and comment the previous line
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
#To run the model on GPU
#model.cuda()   

In the Bert Tokenizer all the words are linked with ids and we can find those ids using Tokenizer It contains around 30,000 word and each related with ids and the mechanism that bert follows to get ids of unknown word is to break them into subwords and then find ids of each subword for eg:beddings will be break down into bed + ##dings except The first word  all are followed by ## token and even if the subwords are unknown it breaks into even characters!

In [ ]:
#Importing the dataset
import pandas as pd
 
dataset=pd.read_csv('/content/50_tweets_per_user.csv')
#dataset.iloc[:,0:1].fillna('other', inplace=True)
y=dataset.iloc[:,1].values

In [ ]:
k=0
X_train=[]
y_train=[]
for i in range(0,50):
  for j in range(k,k+45):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=60

In [ ]:
k=45
X_test=[]
y_test=[]
for i in range(0,50):
  for j in range(k,k+5):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=60

In [ ]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

In [ ]:
y_test

In [ ]:
train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

In [ ]:
np.random.shuffle(train)
np.random.shuffle(test)

In [ ]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower()
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>#@')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(rem_num)  
  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  stem_words=[stemmer.stem(w) for w in filtered_words]
  lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return " ".join(filtered_words)


train.iloc[:,0]=train.iloc[:,0].map(lambda s:preprocess(s))
test.iloc[:,0]=test.iloc[:,0].map(lambda s:preprocess(s))

In [ ]:
X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]

In [ ]:
y_train.shape

(2250,)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [ ]:
X_train=pd.DataFrame(X_train)

In [ ]:
tokenized = X_train.iloc[:,0].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

As BERT is a pretrained model We are required to:

1. Add special tokens to the start and end of each sentence.
2. Pad & truncate all sentences to a single constant length.
3. Explicitly differentiate real tokens from padding tokens with the "attention mask".

In [ ]:

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
#for adding paddings
input_ids = torch.tensor(np.array(padded))


In [ ]:
#to set the paddings to zero and rest to 1
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2250, 36)

In [ ]:
#To convert the parameters to torch tensors
#input_ids = (torch.tensor(padded)).to(device)  
#attention_mask = (torch.tensor(attention_mask)).to(device)
input_ids = (torch.tensor(padded))
attention_mask = (torch.tensor(attention_mask))

In [ ]:
with torch.no_grad():
    last_hidden_states_train = model(input_ids,attention_mask)

In [ ]:
last_hidden_states_train[0].shape

torch.Size([2250, 36, 768])

![Illustration of CLS token purpose](http://www.mccormickml.com/assets/BERT/CLS_token_500x606.png)


In [ ]:
#Extracting the CLS only to feed into the classifier
#features = last_hidden_states[0][:,0,:].cpu().numpy()

X_train = last_hidden_states_train[0][:,0,:].numpy()




In [ ]:
#Normalizing the input for fast execution
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)


In [ ]:
X_train.shape

(2250, 768)

In [ ]:
#LStm expects array of 3-dimension
X_train = np.reshape(X_train, (2250,1,768))

In [ ]:
y_train.shape

(2250, 50)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X_train,y_train, test_size = 0.05, random_state = 0)

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [ ]:
classifier = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
classifier.add(LSTM(units = 48, activation='sigmoid', input_shape = (1, 768),return_sequences = True))
classifier.add(Dropout(0.2))
classifier.add(LSTM(units = 25))
classifier.add(Dropout(0.2))

# Adding the output layer
classifier.add(Dense(units =50,activation='softmax'))



In [ ]:
# Compiling the RNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 48)             156864    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 48)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                7400      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1300      
Total params: 165,564
Trainable params: 165,564
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier.fit(np.array(X_train),np.array(y_train),batch_size=128,epochs=50,validation_data=(np.array(X_eval),np.array(y_eval)),verbose=1,callbacks=[es])

Train on 2137 samples, validate on 113 samples
Epoch 1/50
2137/2137 [==============================] - 3s 1ms/step - loss: 3.9156 - accuracy: 0.0182 - val_loss: 3.9177 - val_accuracy: 0.0088
Epoch 2/50
2137/2137 [==============================] - 0s 102us/step - loss: 3.9117 - accuracy: 0.0215 - val_loss: 3.9144 - val_accuracy: 0.0177
Epoch 3/50
2137/2137 [==============================] - 0s 103us/step - loss: 3.9106 - accuracy: 0.0215 - val_loss: 3.9146 - val_accuracy: 0.0088
Epoch 4/50
2137/2137 [==============================] - 0s 103us/step - loss: 3.9100 - accuracy: 0.0239 - val_loss: 3.9143 - val_accuracy: 0.0000e+00
Epoch 5/50
2137/2137 [==============================] - 0s 105us/step - loss: 3.9079 - accuracy: 0.0370 - val_loss: 3.9096 - val_accuracy: 0.0442
Epoch 6/50
2137/2137 [==============================] - 0s 102us/step - loss: 3.9040 - accuracy: 0.0328 - val_loss: 3.9125 - val_accuracy: 0.0177
Epoch 7/50
2137/2137 [==============================] - 0s 104us/step - los

In [ ]:
X_test=pd.DataFrame(X_test)
tokenized = X_test.iloc[:,0].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

In [ ]:

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(36-len(i)) for i in tokenized.values])

In [ ]:
#for adding paddings
input_ids = torch.tensor(np.array(padded))

In [ ]:
#to set the paddings to zero and rest to 1
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(250, 36)

In [ ]:
#To convert the parameters to torch tensors
#input_ids = (torch.tensor(padded)).to(device)  
#attention_mask = (torch.tensor(attention_mask)).to(device)
input_ids = (torch.tensor(padded))
attention_mask = (torch.tensor(attention_mask))

In [ ]:
with torch.no_grad():
    last_hidden_states_test = model(input_ids,attention_mask)

In [ ]:
X_test= last_hidden_states_test[0][:,0,:].numpy()
X_test=sc.transform(X_test)


In [ ]:
X_test=np.reshape(X_test,(250,1,768))
y_pred =  classifier.predict(X_test)

In [ ]:
#TO get maximum value as 1 and rest to zero
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

In [ ]:
y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

In [ ]:
result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)





In [ ]:
predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)



In [ ]:
print(result)
print(predicted)

[9, 14, 40, 19, 26, 14, 2, 37, 29, 11, 47, 48, 12, 36, 33, 37, 19, 12, 44, 3, 0, 27, 32, 20, 35, 28, 49, 19, 48, 0, 29, 44, 16, 36, 41, 15, 43, 21, 43, 2, 42, 34, 24, 35, 40, 37, 32, 39, 11, 49, 41, 22, 33, 39, 49, 41, 3, 15, 5, 12, 5, 47, 45, 22, 38, 46, 18, 0, 41, 39, 4, 13, 5, 30, 6, 20, 25, 23, 26, 42, 38, 47, 34, 8, 48, 34, 2, 18, 1, 17, 18, 46, 47, 7, 5, 13, 33, 20, 36, 17, 48, 7, 3, 26, 32, 44, 27, 45, 9, 38, 15, 42, 1, 6, 7, 15, 30, 14, 44, 28, 8, 22, 29, 6, 1, 32, 8, 43, 9, 4, 1, 13, 30, 30, 43, 44, 24, 25, 12, 7, 41, 11, 30, 36, 39, 37, 4, 49, 46, 46, 43, 35, 26, 40, 23, 16, 10, 17, 32, 40, 27, 24, 34, 28, 31, 48, 14, 17, 20, 6, 10, 1, 4, 9, 18, 0, 7, 2, 42, 46, 21, 27, 6, 22, 11, 18, 13, 15, 13, 20, 24, 25, 3, 45, 25, 45, 28, 34, 10, 23, 31, 21, 31, 23, 27, 31, 16, 38, 12, 16, 33, 29, 39, 35, 36, 33, 14, 17, 45, 19, 2, 5, 24, 42, 35, 23, 26, 21, 11, 19, 29, 9, 8, 8, 37, 10, 10, 4, 21, 40, 25, 31, 28, 38, 22, 3, 0, 16, 47, 49]
[5, 14, 40, 9, 47, 13, 25, 25, 47, 5, 38, 42, 13,

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

cm

array([[4, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 2, 0, 0]])

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted)) 


Confusion Matrix :
[[4 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 2 0 0]]
Accuracy Score : 0.12
Report : 
              precision    recall  f1-score   support

           0       0.17      0.80      0.28         5
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         5
           5       0.11      0.40      0.17         5
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         5
           8       0.25      0.40      0.31         5
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         5
          13       0.00      0.00      0.00         5
          14      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
